In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#https://github.com/SKTBrain/KoBERT 의 파일들을 Colab으로 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install transformers
!pip3 install kobert-transformers

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-bl1n7zm1/kobert-tokenizer_7f47de56dd4b43b6b0a0c7ab11c7c148
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-bl1n7zm1/kobert-tokenizer_7f47de56dd4b43b6b0a0c7ab11c7c148
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=6868241b833065e9f53f36b110070983df56593e980a1933695a0037bcacb1bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-_g1lj0v4/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.3 MB/s eta 0:00:00


In [3]:
!pip install gluonnlp
!pip install mxnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661674 sha256=607ae74cbc9467ac6d170ab7360b7faad1ff4e61f777e5070026f29fa550931d
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 38.4 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [4]:
# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

device = torch.device("cuda:0")

In [5]:
import os
import ast
from tqdm import tqdm
from collections import Counter

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [6]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU가 사용 중입니다.")
else:
    device = torch.device("cpu")

GPU가 사용 중입니다.


In [7]:
alldf = pd.read_csv('/content/drive/MyDrive/에어비엔비/리뷰번역완료/지은/total/sentiAnalysisAll2.csv')
alldf['MaxEmotion'].value_counts()

기쁨        205180
일상적인      151338
고마운        84259
슬픔         21051
설레는        19635
생각이 많은     16086
짜증남         9605
즐거운         7310
걱정스러운       2944
사랑하는         640
힘듦            87
Name: MaxEmotion, dtype: int64

In [8]:
alldf['MaxEmotionLabel'].value_counts()

2     205180
7     151338
1      84259
6      21051
5      19635
4      16086
9       9605
8       7310
0       2944
3        640
10        87
Name: MaxEmotionLabel, dtype: int64

In [9]:
data = alldf[['text', 'MaxEmotionLabel']]
data

,text,MaxEmotionLabel
0,더할 나위 없이 아파트 자체가 훌륭하고 필요한 모든 것이 갖춰져 있습니다,2
1,게다가 동네 자체가 매우 친절하다는 점 외에도 가족 생활의 에너지가 가득합니다 슈퍼...,2
2,처음부터 집처럼 느껴질 것입니다 쉽게 구할 수 있는 물건을 포장하다,7
3,아파트는 사진에서 보이는 것보다 더 넓습니다,7
4,커플에게 적합합니다 깨끗하고 잘 관리되어 있고 안전하며 유용한 정보가 많이 있습니다,7
...,...,...
518130,도로 옆에 있는 건물이었습니다만 밤의 소음은 신경이 쓰이지 않았습니다,7
518131,무엇보다 호스트 쪽의 대응이 훌륭하고 여행의 질이 올랐습니다,2
518132,꼭 또 이용하고 싶습니다,8
518133,매우 편안한 방이었습니다,2


In [10]:
data_list = []
for ques, label in zip (data['text'], data['MaxEmotionLabel']):
  data = []
  data.append(ques)
  data.append(str(label))
  data_list.append(data)
data_list[:10]

[['더할 나위 없이 아파트 자체가 훌륭하고 필요한 모든 것이 갖춰져 있습니다', '2'],
 ['게다가 동네 자체가 매우 친절하다는 점 외에도 가족 생활의 에너지가 가득합니다 슈퍼마켓과 일부 편의점 및 기차역이 정말 가깝습니다',
  '2'],
 ['처음부터 집처럼 느껴질 것입니다 쉽게 구할 수 있는 물건을 포장하다', '7'],
 ['아파트는 사진에서 보이는 것보다 더 넓습니다', '7'],
 ['커플에게 적합합니다 깨끗하고 잘 관리되어 있고 안전하며 유용한 정보가 많이 있습니다', '7'],
 ['서양인으로서 우리는 다다미 바닥에서 이상한 냄새가 난다는 것을 알았지만 나중에 모든 다다미 방에서 같은 냄새가 난다는 것을 알게 되었습니다 지하철역에서 도보로 약 분 거리에 위치 이 노선에서는 패스가 유효하지 않으며 슈퍼마켓 상점과 레스토랑에서 도보 거리에 있습니다',
  '4'],
 ['전반적으로 좋은 경험을 했으며 도쿄를 여행하는 모든 사람에게 추천합니다', '2'],
 ['아파트는 커플에게 적합합니다 조금 작지만 정말 완벽한 가구가 그것을 보완합니다 일상 생활에 필요한 모든 것이 제공됩니다', '7'],
 ['우리는 또한 동네를 많이 즐겼으며 주요 도시와 가까운 매력적인 장소입니다 머무르게 해주셔서 감사합니다', '1'],
 ['다시 방문하겠습니다', '1']]

In [11]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

In [12]:
len(dataset_train), len(dataset_test)

(414508, 103627)

In [13]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab = vocab, pad = pad, pair = pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [14]:
from kobert_tokenizer import KoBERTTokenizer
import torch
from transformers import BertModel
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [15]:
# parameter 값 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #* epoch 조절해야함
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [17]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)


In [18]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 11,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [19]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [20]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


In [21]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [22]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

  0%|          | 0/6477 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.3789727687835693 train acc 0.171875
epoch 1 batch id 201 loss 1.756592035293579 train acc 0.29267723880597013
epoch 1 batch id 401 loss 1.4080740213394165 train acc 0.41076995012468825
epoch 1 batch id 601 loss 0.9647253155708313 train acc 0.5005199667221298
epoch 1 batch id 801 loss 0.8315761089324951 train acc 0.5567649812734082
epoch 1 batch id 1001 loss 0.6700488924980164 train acc 0.5944992507492507
epoch 1 batch id 1201 loss 0.5223709940910339 train acc 0.621604392173189
epoch 1 batch id 1401 loss 0.7112674713134766 train acc 0.6417848857958601
epoch 1 batch id 1601 loss 0.7275987863540649 train acc 0.6569038881948782
epoch 1 batch id 1801 loss 0.7374564409255981 train acc 0.6702352859522488
epoch 1 batch id 2001 loss 0.43370044231414795 train acc 0.6810891429285357
epoch 1 batch id 2201 loss 0.6562231779098511 train acc 0.6905099954566106
epoch 1 batch id 2401 loss 0.6419888734817505 train acc 0.6984915139525197
epoch 1 batch id 2601 loss 0.507327020168

  0%|          | 0/1620 [00:00<?, ?it/s]

epoch 1 test acc 0.8213383838383838


  0%|          | 0/6477 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8606823442760942


  0%|          | 0/6477 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.11852750182151794 train acc 0.9375
epoch 5 batch id 201 loss 0.20128466188907623 train acc 0.9509483830845771
epoch 5 batch id 401 loss 0.14406652748584747 train acc 0.9518391521197007
epoch 5 batch id 601 loss 0.1028408482670784 train acc 0.9520330698835274
epoch 5 batch id 801 loss 0.00963335856795311 train acc 0.952130149812734
epoch 5 batch id 1001 loss 0.0448797233402729 train acc 0.952125999000999
epoch 5 batch id 1201 loss 0.11990951001644135 train acc 0.9528387801831807
epoch 5 batch id 1401 loss 0.12074730545282364 train acc 0.9525227516059958
epoch 5 batch id 1601 loss 0.22534988820552826 train acc 0.952432073703935
epoch 5 batch id 1801 loss 0.12363483756780624 train acc 0.9525176985008329
epoch 5 batch id 2001 loss 0.060326702892780304 train acc 0.9530625312343828
epoch 5 batch id 2201 loss 0.11650527268648148 train acc 0.9531675942753294
epoch 5 batch id 2401 loss 0.16835792362689972 train acc 0.9534894314868805
epoch 5 batch id 2601 loss 0.029426

  0%|          | 0/1620 [00:00<?, ?it/s]

epoch 5 test acc 0.8629585788439955


In [24]:
torch.save(model, f'/content/drive/MyDrive/에어비앤비2/huggingface_multiSentiment/SentimentAnalysisKOBert.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/에어비앤비2/huggingface_multiSentiment/SentimentAnalysisKOBert_StateDict.pt')

FileNotFoundError: ignored

In [25]:
import torch.nn as nn
import torch
#model = torch.load('/content/drive/MyDrive/에어비앤비2/huggingface_multiSentiment/SentimentAnalysisKOBert_StateDict.pt')
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [26]:
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=4)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [27]:
sentence = '데이터 적어서 과적합 난 듯' #*
predict(sentence)

6

In [28]:
sentence = '숙소가 좁고 벌레가 나왔다'
predict(sentence)

9

In [29]:
sentence = '냄새는 안났지만 숙소는 좁았다.'
predict(sentence)

6

In [30]:
sentence = '카페모카는 너무 맛있었는데 추웠다.'
predict(sentence)

6

In [31]:
sentence = '병원의 주요 커버 범위를 시각화하였다.'
predict(sentence)

7